In [1]:
import psycopg2
import pandas as pd
import os
import csv

In [13]:
# getting the password for the database from an environment variable
db_pw = os.environ.get('DB_PASS')

In [14]:
# Connecting to the Postgres database
try:
    conn = psycopg2.connect(f"host=localhost user=postgres dbname=postgres password={db_pw}")
except psycopg2.Error as e:
    print('Error: No Conection to database possible.')
    print(e)

In [15]:
# Creating a cursor object for executing the SQL queries
try:
    cur = conn.cursor()
except psycopg2.Error as e:
    print(e)

In [16]:
# this setting reduces coding a commit for every query
conn.set_session(autocommit=True)

In [17]:
# Creating the Database for the NFL data
try:
    cur.execute("CREATE DATABASE NFL_Scores_Bets")
except psycopg2.Error as e:
    print(e)

FEHLER:  Datenbank »nfl_scores_bets« existiert bereits



In [18]:
# Closing of the connection to the postgres database
try:
    conn.close()
except psycopg2.Error as e:
    print(e)

# Connecting to the new database nfl_scores_bets    
try:
    conn = psycopg2.connect(f"host=localhost user=postgres dbname=nfl_scores_bets password={db_pw}")
except psycopg2.Error as e:
    print('Error: Cnnection to database failed')
    print(e)
    
try:
    cur = conn.cursor()
except psycopg2.Error as e:
    print('Cursor failed')
    print(e)
    
conn.set_session(autocommit=True)

# Creating the 3 tables stadiums, teams and scores_bets

In [152]:
try:
    cur.execute("CREATE TABLE IF NOT EXISTS stadiums \
    (stadium_name varchar NOT NULL, \
    stadium_location varchar, \
    stadium_open_year int, \
    stadium_close_year int, \
    stadium_type varchar, \
    stadium_address varchar, \
    stadium_weather_station_code varchar, \
    stadium_weather_type varchar, \
    stadium_capacity int, \
    stadium_surface varchar, \
    PRIMARY KEY (stadium_name))")
except psycopg2.Error as e:
    print('Error: Creating table nfl_stadiums failed')
    print(e)

In [20]:
try:
    cur.execute("CREATE TABLE IF NOT EXISTS teams \
    (team_name varchar PRIMARY KEY, \
    team_name_short varchar, \
    team_id varchar NOT NULL, \
    team_id_pfr varchar, \
    team_conference varchar, \
    team_division varchar, \
    team_conference_pre2002 varchar, \
    team_division_pre2002 varchar)")
except psycopg2.Error as e: 
    print('Error: Creating table teams failed')
    print(e)

In [130]:
# schedule_week must be varchar because there are also texts (Superbwl, Wildcard ....)
try:
    cur.execute("CREATE TABLE IF NOT EXISTS scores_bets \
    (id SERIAL PRIMARY KEY, \
    schedule_date date, \
    schedule_season varchar, \
    schedule_week varchar, \
    schedule_playoff bool, \
    team_home varchar, \
    score_home smallint, \
    score_away smallint, \
    team_away varchar, \
    team_favorite_id varchar, \
    spread_favorite varchar, \
    over_under_line varchar, \
    stadium_name varchar, \
    stadium_neutral bool, \
    weather_temperature numeric, \
    weather_wind_mph numeric, \
    weather_humidity numeric, \
    weather_detail varchar, \
    foreign key (team_home) references teams(team_name), \
    foreign key (team_away) references teams(team_name))")

except psycopg2.Error as e: 
    print('Error: Creating table scores_bets failed')
    print(e)

# Loading of the csv files

In [44]:
teams = pd.read_csv('data/teams.csv')

In [45]:
teams.head()

,team_name,team_name_short,team_id,team_id_pfr,team_conference,team_division,team_conference_pre2002,team_division_pre2002
0,Arizona Cardinals,Cardinals,ARI,CRD,NFC,NFC West,NFC,NFC West
1,Atlanta Falcons,Falcons,ATL,ATL,NFC,NFC South,NFC,NFC West
2,Baltimore Colts,Colts,IND,CLT,AFC,NaN,AFC,AFC East
3,Baltimore Ravens,Ravens,BAL,RAV,AFC,AFC North,AFC,AFC Central
4,Boston Patriots,Patriots,NE,NWE,AFC,NaN,AFC,NaN


In [76]:
scores_bets = pd.read_csv('data/scores_bets.csv')

In [86]:
scores_bets.tail()

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail
13499,2023-01-08,2022,18,False,Philadelphia Eagles,NaN,NaN,New York Giants,NaN,NaN,NaN,Lincoln Financial Field,False,NaN,NaN,NaN,NaN
13500,2023-01-08,2022,18,False,Pittsburgh Steelers,NaN,NaN,Cleveland Browns,NaN,NaN,NaN,Acrisure Stadium,False,NaN,NaN,NaN,NaN
13501,2023-01-08,2022,18,False,San Francisco 49ers,NaN,NaN,Arizona Cardinals,NaN,NaN,NaN,Levi's Stadium,False,NaN,NaN,NaN,NaN
13502,2023-01-08,2022,18,False,Seattle Seahawks,NaN,NaN,Los Angeles Rams,NaN,NaN,NaN,Lumen Field,False,NaN,NaN,NaN,NaN
13503,2023-01-08,2022,18,False,Washington Commanders,NaN,NaN,Dallas Cowboys,NaN,NaN,NaN,FedEx Field,False,NaN,NaN,NaN,NaN


In [78]:
# changing the dateformat from dd/mm/yyyy to yyyy-mm-dd for postgres insert
scores_bets['schedule_date'] = pd.to_datetime(scores_bets['schedule_date'])
scores_bets.head()

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail
0,1966-09-02,1966,1,False,Miami Dolphins,14.0,23.0,Oakland Raiders,NaN,NaN,NaN,Orange Bowl,False,83.0,6.0,71.0,NaN
1,1966-09-03,1966,1,False,Houston Oilers,45.0,7.0,Denver Broncos,NaN,NaN,NaN,Rice Stadium,False,81.0,7.0,70.0,NaN
2,1966-09-04,1966,1,False,San Diego Chargers,27.0,7.0,Buffalo Bills,NaN,NaN,NaN,Balboa Stadium,False,70.0,7.0,82.0,NaN
3,1966-09-09,1966,2,False,Miami Dolphins,14.0,19.0,New York Jets,NaN,NaN,NaN,Orange Bowl,False,82.0,11.0,78.0,NaN
4,1966-09-10,1966,1,False,Green Bay Packers,24.0,3.0,Baltimore Colts,NaN,NaN,NaN,Lambeau Field,False,64.0,8.0,62.0,NaN


In [87]:
# Replacing NaN Values in the Dataframe to None. 
# Otherwise the columns can´t get insert into Postgres 
scores_bets = scores_bets.where(pd.notnull(scores_bets), None)
scores_bets.tail()

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail
13499,2023-01-08,2022,18,False,Philadelphia Eagles,None,None,New York Giants,None,None,None,Lincoln Financial Field,False,None,None,None,None
13500,2023-01-08,2022,18,False,Pittsburgh Steelers,None,None,Cleveland Browns,None,None,None,Acrisure Stadium,False,None,None,None,None
13501,2023-01-08,2022,18,False,San Francisco 49ers,None,None,Arizona Cardinals,None,None,None,Levi's Stadium,False,None,None,None,None
13502,2023-01-08,2022,18,False,Seattle Seahawks,None,None,Los Angeles Rams,None,None,None,Lumen Field,False,None,None,None,None
13503,2023-01-08,2022,18,False,Washington Commanders,None,None,Dallas Cowboys,None,None,None,FedEx Field,False,None,None,None,None


In [3]:
stadiums = pd.read_csv('data/stadiums.csv', encoding='latin1')
stadiums.head()

,stadium_name,stadium_location,stadium_open,stadium_close,stadium_type,stadium_address,stadium_weather_station_code,stadium_weather_type,stadium_capacity,stadium_surface,STATION,NAME,LATITUDE,LONGITUDE,ELEVATION
0,Acrisure Stadium,"Pittsburgh, PA",2001.0,NaN,outdoor,"100 Art Rooney Ave, Pittsburgh, PA 15212",15212,cold,"65,500",Grass,USW00094823,"PITTSBURGH ASOS, PA US",40.4846,-80.2144,366.7
1,Alamo Dome,"San Antonio, TX",NaN,NaN,indoor,"100 Montana St, San Antonio, TX 78203",78203,dome,72000,FieldTurf,NaN,NaN,NaN,NaN,NaN
2,Allegiant Stadium,"Paradise, NV",2020.0,NaN,indoor,NaN,NaN,dome,65000,Grass,NaN,NaN,NaN,NaN,NaN
3,Allianz Arena,"Munich, Germany",NaN,NaN,outdoor,NaN,NaN,moderate,"75,024",Grass,NaN,NaN,NaN,NaN,NaN
4,Alltel Stadium,"Jacksonville, FL",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
stadiums = stadiums.where(pd.notnull(stadiums), None)
stadiums.head()

,stadium_name,stadium_location,stadium_open,stadium_close,stadium_type,stadium_address,stadium_weather_station_code,stadium_weather_type,stadium_capacity,stadium_surface,STATION,NAME,LATITUDE,LONGITUDE,ELEVATION
0,Acrisure Stadium,"Pittsburgh, PA",2001,None,outdoor,"100 Art Rooney Ave, Pittsburgh, PA 15212",15212,cold,"65,500",Grass,USW00094823,"PITTSBURGH ASOS, PA US",40.4846,-80.2144,366.7
1,Alamo Dome,"San Antonio, TX",None,None,indoor,"100 Montana St, San Antonio, TX 78203",78203,dome,72000,FieldTurf,None,None,None,None,None
2,Allegiant Stadium,"Paradise, NV",2020,None,indoor,None,None,dome,65000,Grass,None,None,None,None,None
3,Allianz Arena,"Munich, Germany",None,None,outdoor,None,None,moderate,"75,024",Grass,None,None,None,None,None
4,Alltel Stadium,"Jacksonville, FL",None,None,None,None,None,None,None,None,None,None,None,None,None


In [177]:
# Deleting the unnessesary rows
stadiums = stadiums[['stadium_name', 'stadium_location', 'stadium_open', 'stadium_close', 'stadium_type', 'stadium_address', 'stadium_weather_station_code', 'stadium_weather_type', 'stadium_capacity', 'stadium_surface']]
stadiums['stadium_capacity'] = stadiums['stadium_capacity'].str.replace(',','')
stadiums.head()

,stadium_name,stadium_location,stadium_open,stadium_close,stadium_type,stadium_address,stadium_weather_station_code,stadium_weather_type,stadium_capacity,stadium_surface
0,Acrisure Stadium,"Pittsburgh, PA",2001,None,outdoor,"100 Art Rooney Ave, Pittsburgh, PA 15212",15212,cold,65500,Grass
1,Alamo Dome,"San Antonio, TX",None,None,indoor,"100 Montana St, San Antonio, TX 78203",78203,dome,72000,FieldTurf
2,Allegiant Stadium,"Paradise, NV",2020,None,indoor,None,None,dome,65000,Grass
3,Allianz Arena,"Munich, Germany",None,None,outdoor,None,None,moderate,75024,Grass
4,Alltel Stadium,"Jacksonville, FL",None,None,None,None,None,None,None,None


# Insert the data from the dataframes into the database

In [178]:
stadium_table_insert = ("""INSERT INTO stadiums (
stadium_name,
stadium_location,
stadium_open_year,
stadium_close_year,
stadium_type,
stadium_address, 
stadium_weather_station_code,
stadium_weather_type,
stadium_capacity,
stadium_surface) 
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
""")

In [147]:
teams_table_insert = ("""INSERT INTO teams (
team_name,
team_name_short,
team_id,
team_id_pfr,
team_conference,
team_division,
team_conference_pre2002,
team_division_pre2002)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
""")

In [148]:
scores_bets_table_insert = ("""INSERT INTO scores_bets (
schedule_date,
schedule_season,
schedule_week,
schedule_playoff,
team_home,
score_home,
score_away,
team_away,
team_favorite_id,
spread_favorite,
over_under_line,
stadium_name,
stadium_neutral,
weather_temperature,
weather_wind_mph,
weather_humidity,
weather_detail) 
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
""")

In [179]:
for i, row in stadiums.iterrows():
    cur.execute(stadium_table_insert, list(row))

In [128]:
for i, row in teams.iterrows():
    cur.execute(teams_table_insert, list(row))

UniqueViolation: FEHLER:  doppelter Schlüsselwert verletzt Unique-Constraint »teams_pkey«
DETAIL:  Schlüssel »(team_name)=(Arizona Cardinals)« existiert bereits.


In [131]:
#for i, row in scores_bets.iterrows():
#    cur.execute(scores_bets_table_insert, list(row))